<a href="https://colab.research.google.com/github/sunyeul/ToyProjectLab/blob/feature%2Fnanogpt_tutorial/nanoGPT/bigram_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# データセットをダウンロードしましょう。
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-04-23 16:41:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.007s  

2023-04-23 16:41:13 (160 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from tqdm.auto import tqdm
from dataclasses import dataclass

torch.manual_seed(3655)


@dataclass
class Config:
    vocab_size: int = 65
    batch_size: int = 8
    block_size: int = 16 # what is the maximum context length for predictions?

    train_size: float = 0.8  # valid_sizeは自動で0.2に決まる
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))
print(text[:1_000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [ ]:
# テキストから重複を除いた文字列を取得し、アルファベット順にソートする
chars = sorted(list(set(text)))
print("".join(chars))

# ボキャブラリーのサイズを取得する
vocab_size = len(chars)
Config.vocab_size = vocab_size
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# 文字列をインデックスに変換するための辞書を作成する
# s2iは文字列をインデックスに変換するための辞書
s2i = {ch:i for i, ch in enumerate(chars)}

# インデックスを文字列に変換するための辞書を作成する
# i2sはインデックスを文字列に変換するための辞書
i2s = {i:ch for i, ch in enumerate(chars)}

# 文字列を数値のリストに変換する関数を定義する
encode = lambda s: [s2i[c] for c in s]

# 数値のリストを文字列に変換する関数を定義する
decode = lambda l: ''.join([i2s[i] for i in l])

In [ ]:
# テキストを数値のリストに変換する
data = torch.tensor(encode(text), dtype=torch.long)

# データの形状とデータ型を表示する
print(data.shape, data.dtype)

# 先頭の1000文字を表示する
print(data[:1000]) # GPTにとっては、ここで表示される1000文字は以下のようになる

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
# 学習用データと検証用データに分割する
n = int(Config.train_size * len(data))

train_data = data[:n]
val_data = data[n:]

In [ ]:
# 学習用データを最初のConfig.block_size文字だけに限定する
x = train_data[:Config.block_size]

# 学習用データを2文字目から最初のConfig.block_size+1文字に限定する
y = train_data[1:Config.block_size+1]

# Config.block_size回繰り返す
for t in range(Config.block_size):

    # 入力となる文字列を1文字からt+1文字までに限定する
    context = x[:t+1]

    # 正解の文字を取得する
    target = y[t]

    # 入力がcontextのときに、正解がtargetであることを表示する
    print(f"入力が{context}のときに、正解は{target}です")

入力がtensor([18])のときに、正解は47です
入力がtensor([18, 47])のときに、正解は56です
入力がtensor([18, 47, 56])のときに、正解は57です
入力がtensor([18, 47, 56, 57])のときに、正解は58です
入力がtensor([18, 47, 56, 57, 58])のときに、正解は1です
入力がtensor([18, 47, 56, 57, 58,  1])のときに、正解は15です
入力がtensor([18, 47, 56, 57, 58,  1, 15])のときに、正解は47です
入力がtensor([18, 47, 56, 57, 58,  1, 15, 47])のときに、正解は58です
入力がtensor([18, 47, 56, 57, 58,  1, 15, 47, 58])のときに、正解は47です
入力がtensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])のときに、正解は64です
入力がtensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64])のときに、正解は43です
入力がtensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43])のときに、正解は52です
入力がtensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52])のときに、正解は10です
入力がtensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10])のときに、正解は0です
入力がtensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0])のときに、正解は14です
入力がtensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14])のときに、正解は43です


In [ ]:
def get_batch(split):
    # 入力と正解の小さなバッチを生成する
    # splitが'train'の場合は学習用データから、'val'の場合は検証用データからデータを取得する
    data = train_data if split == 'train' else val_data
    
    # バッチを開始するためのランダムなインデックスを生成する
    idx = torch.randint(high=len(data) - Config.block_size, size=(Config.batch_size,))
    
    # xは、block_sizeの長さのシーケンスのバッチである
    x = torch.stack([data[i:i+Config.block_size] for i in idx])  # [batch_size, block_size]
    
    # yは、xと同じものであるが、1つずつずれている
    y = torch.stack([data[i+1:i+Config.block_size+1] for i in idx])  # [batch_size, block_size]
    
    return x.to(Config.device), y.to(Config.device)

In [ ]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([8, 16])
tensor([[59, 43, 43, 52, 12,  0,  0, 29, 33, 17, 17, 26,  1, 25, 13, 30],
        [ 1, 63, 53, 59, 56,  1, 55, 59, 43, 43, 52,  6,  0, 13, 52, 42],
        [ 0, 18, 39, 56, 43, 61, 43, 50, 50,  2,  1, 19, 53, 42,  1, 49],
        [57, 43, 42,  1, 61, 47, 58, 46,  1, 58, 46, 43,  1, 40, 50, 53],
        [40, 39, 41, 49, 10,  0, 32, 46, 43, 56, 43,  1, 47, 57,  1, 39],
        [ 1, 49, 47, 52, 45, 11,  1, 39, 52, 42,  1, 52, 53, 58,  1, 58],
        [ 1, 58, 46, 53, 59,  1, 46, 39, 57, 58,  1, 52, 53,  1, 41, 39],
        [56, 53, 59, 45, 46, 58,  1, 44, 53, 56, 58, 46,  1, 50, 43, 57]],
       device='cuda:0')
targets:
torch.Size([8, 16])
tensor([[43, 43, 52, 12,  0,  0, 29, 33, 17, 17, 26,  1, 25, 13, 30, 19],
        [63, 53, 59, 56,  1, 55, 59, 43, 43, 52,  6,  0, 13, 52, 42,  1],
        [18, 39, 56, 43, 61, 43, 50, 50,  2,  1, 19, 53, 42,  1, 49, 52],
        [43, 42,  1, 61, 47, 58, 46,  1, 58, 46, 43,  1, 40, 50, 53, 53],
        [39, 41, 49, 10,  0, 3

In [ ]:
for b in range(Config.batch_size): # batch dimension
    for t in range(Config.block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [59] the target: 43
when input is [59, 43] the target: 43
when input is [59, 43, 43] the target: 52
when input is [59, 43, 43, 52] the target: 12
when input is [59, 43, 43, 52, 12] the target: 0
when input is [59, 43, 43, 52, 12, 0] the target: 0
when input is [59, 43, 43, 52, 12, 0, 0] the target: 29
when input is [59, 43, 43, 52, 12, 0, 0, 29] the target: 33
when input is [59, 43, 43, 52, 12, 0, 0, 29, 33] the target: 17
when input is [59, 43, 43, 52, 12, 0, 0, 29, 33, 17] the target: 17
when input is [59, 43, 43, 52, 12, 0, 0, 29, 33, 17, 17] the target: 26
when input is [59, 43, 43, 52, 12, 0, 0, 29, 33, 17, 17, 26] the target: 1
when input is [59, 43, 43, 52, 12, 0, 0, 29, 33, 17, 17, 26, 1] the target: 25
when input is [59, 43, 43, 52, 12, 0, 0, 29, 33, 17, 17, 26, 1, 25] the target: 13
when input is [59, 43, 43, 52, 12, 0, 0, 29, 33, 17, 17, 26, 1, 25, 13] the target: 30
when input is [59, 43, 43, 52, 12, 0, 0, 29, 33, 17, 17, 26, 1, 25, 13, 30] the target: 19
when

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        # トークンの埋め込み表を作成する
        self.token_embedding_table = nn.Embedding(config.vocab_size, config.vocab_size, device=Config.device)

    def forward(self, idx, targets=None):
        """
        バイグラム言語モデルの順伝播関数
        """
        # logitsは、モデルのログオッズを含む形状が[B、T、C]のテンソルです
        logits = self.token_embedding_table(idx)  # [B, T, C]

        if targets is None:
            loss = None
        else:
            # [B, T, C]
            B, T, C = logits.shape
            # [B*T, C]
            logits = logits.view(B*T, C)
            # [B*T]
            targets = targets.view(B*T)
            # lossは、クロスエントロピー損失を示す
            loss = F.cross_entropy(logits, targets)  # loss is the cross entropy loss
        return logits, loss

    def generate(self, idx, max_new_tokens):
        """
        バイグラム言語モデルの生成関数
        """
        # idxは、現在の文脈のインデックスの(B, T)配列である
        for _ in range(max_new_tokens):
            # 予測を取得する
            logits, _ = self(idx)  # targets is None
            # 最後の時間ステップにフォーカスする
            logits = logits[:, -1, :]  # [B, C]
            # 確率を取得するためにsoftmaxを適用する
            probs = F.softmax(logits, dim=-1)  # [B, C]
            # 分布からサンプリングする
            idx_next = torch.multinomial(probs, num_samples=1)  # [B, 1]
            # サンプリングされたインデックスを実行中のシーケンスに追加する
            idx = torch.cat((idx, idx_next), dim=1)  # [B, T+1]
        return idx

In [ ]:
# モデルをインスタンス化する
model = BigramLanguageModel(config=Config)

# 順伝播を実行し、ログオッズと損失を取得する
logits, loss = model(xb, yb)

# logitsの形状と損失を表示する
print(logits.shape)
print(loss)

# モデルによって生成されたテキストを表示する
generated_text = model.generate(idx=torch.zeros((1, 1,), dtype=torch.long, device=Config.device), max_new_tokens=100)[0].tolist()
decoded_text = decode(generated_text)
print(decoded_text)

torch.Size([128, 65])
tensor(4.6667, device='cuda:0', grad_fn=<NllLossBackward0>)

WBiuOhZGs:;:ZxBi-.DoBii3&ssFRP;UpMFqkxoQVj'Szodw,$SqcG zVzUnNrR-quLnDiAYs3dxaC zj.nuSEj;KgxjGmm.,dCf


In [ ]:
# PyTorchのオプティマイザを作成する
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

# 100,000回のステップを実行する
for steps in tqdm(range(100_000)):
    
    # データのバッチをサンプリングする
    xb, yb = get_batch('train')

    # 損失を評価する
    logits, loss = model(xb, yb)

    # 勾配をゼロに設定して、逆伝播を計算する
    optimizer.zero_grad(set_to_none=True)
    loss.backward()

    # パラメータを更新する
    optimizer.step()

# 損失を表示する
print(loss.item())

  0%|          | 0/100000 [00:00<?, ?it/s]

2.518681287765503


In [ ]:
# モデルによって生成されたテキストを表示する
generated_text = model.generate(idx=torch.zeros((1, 1), dtype=torch.long, device=Config.device), max_new_tokens=500)[0].tolist()
decoded_text = decode(generated_text)
print(decoded_text)


F d my cu HEY:
aser s Thio ke'toule touk or!
UFoueal clou bradobl s tht
Bow
Wimennk, otono meeang,
Fang g ponkison?
hcons thy ird tee fu d w LA:
Ye ADUTar ho th atthed oreeonsw, lly ellllyom'lkepl! as t
Ty oor:
I hindeng hathe quris semus,
Fintharngririmead? brd llsceerom, s
Whe s nd!

The h wave t:

Alithede
OOLA:
BE: thithisestuchengen th,
D ier?
AR:
OUCOLIfaishou k p; their Oh 'thsowsan,

Whe,
N byork fom t youpizas?

Ane e,
HANurd

LLED s t wicurtea t br-e he thelll th upostory hak min Tweal
